In [ ]:
import pandas as pd 
import numpy as np 


df = pd.read_csv(r'/home/nader-muhammad/Documents/Data Engineering/Big_Projects/Data_warehouse2_CarsData/The Data/car_prices.csv')
df

### Staging Layer

In [ ]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus


user="dbeaver"
password=quote_plus('Dbeaver@123')
dbname='ETL_cars'
host='localhost'
port='3306'


engine=create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}")


df.to_sql("Bronze_ETL_cars", engine, index=False, if_exists="append")

In [ ]:
df.count()

### ETL Pocessing

### Uniqueness

In [ ]:
df = df.drop_duplicates(subset=['year','make','model','trim','body','transmission', 'state', 'color','interior', 'seller','mmr','sellingprice','saledate'])

### Standardize(Consistency)

In [ ]:
df['year'].unique()

In [ ]:
df['make'].unique()

In [ ]:
df['make'] = df['make'].replace({'Mercedes-B':'Mercedes-Benz', 'Vw':'Volkswagen', 'Ford Tk':'Ford Truck','Mazda Tk':'Mazda Truck', 'Hyundai Tk':'Hyundai Truck', 'Landrover':"Range Rover",'Land Rover':"Range Rover", 
                     'Ram':'RAM', 'Land Rover':'Range Rover','Dodge Tk':'Dodge Truck', 'Gmc':'GMC','Geo':'GEO',})

In [ ]:
df["make"] = df['make'].str.strip().str.title()

In [ ]:
df['make'].unique()

In [ ]:
df['model'] = df['model'].str.title().str.strip().astype(str)

df['model']=df['model'].replace({'Uplandr':"Uplander", "ridgelin":"ridge line", "Sq5":"SQ5","Xc70":"XC70","Rr":"RR","Elr":"ELR", "Rx 350":"RX 350", "Fx":"FX", "Srx":"SRX","Ilx":"ILX","Tts":"TTS",
                                 "Mdx":"DDX","Gs 350":"GS 350","Rio":"RIO", "Es 350":"ES 350","Cx-9":"CX-9","Jx":"JX","Rx 450H":"RX 450H", "Lr4":"LR4","Ls 460":"LS 460","Mkx":"MKX","Mkt":"MKT",
                                  "Xf":"XF","Xc60":"XC-60","Nv":"NV","Cc":"CC",   })




In [ ]:
df['trim'].unique()

In [ ]:
df['body'] = df['body'].str.title().replace({'Cts-V Wagon':'CTS-V Wagon', 'Tsx Sport Wagon':'TSX Sport Wagon', 'Cts Wagon':'CTS Wagon', 'Cts-V Coupe':'CTS-V Coupe', 'Cts Coupe':'CTS Coupe', 'Suv':'SUV'})
df['body'].unique()

In [ ]:
df['transmission'] = df['transmission'].str.title()
df['transmission'].unique()

In [ ]:
df['vin'] = df['vin'].astype(str).apply(lambda x:x[0:3] + '-' + x[4:7] + '-' + x[8:11] + '-' + x[12:])

df

In [ ]:
df['state'].unique()

In [ ]:
df['color'].unique()

In [ ]:
df['color'] = df['color'].str.title().replace('—', np.nan)

df['color'].unique()

In [ ]:
df['interior'].unique()

In [ ]:
df['interior'] = df['interior'].replace('—', np.nan)
df['interior'] = df['interior'].str.title()
df['interior'].unique()

In [ ]:
df["seller"] = df['seller'].str.title()
df

### Handling Nulls(Completeness)

In [ ]:
df.info()

In [ ]:
df['make'] = df['make'].fillna(method='ffill')

In [ ]:
df['model'] = df['model'].fillna("No Model")

In [ ]:
df['trim'] = df['trim'].fillna("No Data")

In [ ]:
df['body'] = df['body'].fillna('No Info')

In [ ]:
df['transmission'] = df['transmission'].fillna(method='bfill')

In [ ]:
df

In [ ]:
df['vin'] = df['vin'].fillna("Missing")

In [ ]:
df['condition'] = df['condition'].fillna(df['condition'].mean())

In [ ]:
df['odometer'] = df['odometer'].fillna(df['odometer'].mean())

In [ ]:
df['color'] = df['color'].fillna('No Color')

In [ ]:
df['interior'] = df['interior'].fillna('No Data')

In [ ]:
df['mmr'] = df['mmr'].fillna(df['mmr'].mean())

In [ ]:
df['sellingprice'] = df['sellingprice'].fillna(df['sellingprice'].mean())

In [ ]:
df['saledate'] = df['saledate'].fillna(method='ffill')

In [ ]:
df.info()

### Convert Data Types(Validity)

In [ ]:
df.info()

In [ ]:
df['condition'] = df['condition'].astype(int)

In [ ]:
df['saledate'] = df['saledate'].astype(str).apply(lambda x:x[0:16])

In [ ]:
df

In [ ]:
df['saledate'] = pd.to_datetime(df['saledate'], format='mixed', errors='coerce')

In [ ]:
df.info()

In [ ]:
# df.to_sql("Silver_ETL_cars", engine, if_exists='append', index=False)

In [ ]:
df.to_sql("Silver_cars", engine, if_exists='append', index=False)